#Decision Tree


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
class Node:
  def __init__(self,feature= None,threshold=None,left=None,right=None,*,value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value


  def is_leaf_node(self):
    return self.value is not None

In [3]:
class DecisionTree:
  def __init__(self,min_samples_split=2,max_depth=100,n_features=None,criterion = 'entropy'):

    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None
    self.criterion = criterion

  def fit(self,X,y):
    self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
    self.root = self._grow_tree(X,y)

  def _grow_tree(self,X,y,depth=0):
    n_samples,n_feats = X.shape
    n_labels = len(np.unique(y))

    if(depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idx = np.random.choice(n_feats,self.n_features,replace = False)

    best_feature,best_thresh= self._best_split(X,y,feat_idx)

    left_idxs,right_idxs = self._split(X[:,best_feature],best_thresh)

    left = self._grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
    right = self._grow_tree(X[right_idxs,:],y[right_idxs],depth+1)

    return Node(best_feature,best_thresh,left,right)

  def _most_common_label(self,y):
    counter = Counter(y)
    value = counter.most_common(1)[0][0]
    return value

  def _best_split(self,X,y,feat_idxs):
    best_gain = -1
    split_idx,split_threshold = None,None

    for feat_idx in feat_idxs:
      X_column = X[:,feat_idx]
      thresholds = np.unique(X_column)
      for threshold in thresholds:
        gain =  self._information_gain(y,X_column,threshold)

        if gain> best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx,split_threshold

  def _information_gain(self,y,X_column,threshold):


    left_idx,right_idx = self._split(X_column,threshold)

    if len(left_idx) == 0 or len(right_idx) == 0:
      return 0


    n = len(y)

    n_l,n_r = len(left_idx),len(right_idx)


    if self.criterion == 'entropy':
        parent = self._entropy(y)
        left = self._entropy(y[left_idx])
        right = self._entropy(y[right_idx])
    elif self.criterion == 'gini':
        parent = self._gini(y)
        left = self._gini(y[left_idx])
        right = self._gini(y[right_idx])
    else:
        raise ValueError("criterion must be 'entropy' or 'gini'")

    child_impurity = (n_l/n)*left + (n_r/n)*right
    return parent - child_impurity

    return parent_entropy - child_entropy

  def _entropy(self,y):
    hist = np.bincount(y)
    ps = hist/len(y)

    return -np.sum([p*np.log(p) for p in ps if p>0])


  def _split(self,X_column,split_thresh):
    left_idxs = np.argwhere(X_column<=split_thresh).flatten()

    right_idxs = np.argwhere(X_column>split_thresh).flatten()

    return left_idxs,right_idxs

  def _gini(self, y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return 1 - np.sum(ps**2)

  def predict(self,X):
    return np.array([self._traverse_tree(x,self.root) for x in X])

  def _traverse_tree(self,x,node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x,node.left)
    return self._traverse_tree(x,node.right)



# RandomForest


In [4]:
from re import T
class RandomForest:
  def __init__(self,n_trees=10,max_depth=10,min_samples_split=2,n_features= None):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.n_features = n_features
    self.trees = []

  def fit(self,X,y):
    self.trees = []

    for _ in range(self.n_trees):
      tree =DecisionTree(max_depth=self.max_depth,min_samples_split=self.min_samples_split,n_features = self.n_features)

      X_sample,y_sample = self._boostrap_samples(X,y)

      tree.fit(X_sample,y_sample)

      self.trees.append(tree)


  def _boostrap_samples(self,X,y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples,n_samples,replace=True)
    return X[idxs],y[idxs]


  def _most_common_label(self,y):
    counter = Counter(y)
    value = counter.most_common(1)[0][0]
    return value

  def predict(self,X):
   preds =  np.array([tree.predict(X) for tree in self.trees])

   tree_preds = np.swapaxes(preds,0,1)

   return np.array([self._most_common_label(pred) for pred in tree_preds])

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X,y = load_breast_cancer(return_X_y=True)

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify = y,test_size=0.2)


clf = RandomForest(n_trees=20)

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test,y_pred))

0.9298245614035088
